In [ ]:
import numpy as np #import numpy
import matplotlib.pyplot as plt #import matplotlib
from matplotlib.pyplot import rcParams
rcParams['mathtext.default'] = 'regular'
import math
from scipy.interpolate import interp1d, splrep, splev
from pylab import setp
import helper

In [ ]:
from scipy.interpolate import interp1d

energy_GeV, aeffsr_m2sr = helper.get_aeff('ehenextgen')
energy_eV = energy_GeV*1E9
energy_eV = energy_eV[1:]
aeffsr_m2sr = aeffsr_m2sr[1:]

# need interpolator for IceCube sensitivity
icecube_interpolator = interp1d(np.log10(energy_eV), np.log10(aeffsr_m2sr),
                              bounds_error=False,
                              fill_value=0.
                              )

# debug
# some_e_bins = np.logspace(15,20,100)
# interp_vals = icecube_interpolator(np.log10(some_e_bins))
# fig2 = plt.figure(figsize=(2*8.5,2*3.5))
# ax = fig2.add_subplot(1,1,1)
# ax.plot(energy_eV, aeffsr_m2sr)
# ax.plot(some_e_bins, 10**interp_vals,'--')
# plt.loglog()


In [ ]:
# fluxes
def power_law(E_eV, phi, gamma):
    return phi * np.power(E_eV/1.E14, gamma)

# some fluxes (returned in number of particles)
def icecube_thrumu_function(E_eV):
    return power_law(E_eV, 3.*1.44E-27, -2.37)
def icecube_hese_function(E_eV):
    return power_law(E_eV, 5.68E-27, -2.87)

# do an integral
bins = np.arange(14,20,0.5)
livetime = 60*60*24*365*12

def run_integral(energy_bins, flux_model, aeff_interp_object):

    energy_bins=[]
    counts = []
    
    for bin in bins:
        temp_logev = np.arange(bin, bin+0.5, 0.01)
        temp_energy = np.power(10., temp_logev)
    
        # the effective area estimate
        temp_aeff = np.power(10., aeff_interp_object(temp_logev))
    
        # the flux estimate
        temp_model = flux_model(temp_energy)
        temp_counts = np.trapz(temp_model*temp_aeff*livetime,temp_energy)
        counts.append(temp_counts)
        energy_bins.append(np.power(10.,bin))

    return energy_bins, counts

energy_bins, counts_thrumu = run_integral(bins, icecube_thrumu_function, icecube_interpolator)
energy_bins, counts_hese = run_integral(bins, icecube_hese_function, icecube_interpolator)

counts_thrumu = np.array(counts_thrumu)
counts_hese = np.array(counts_hese)
energy_bins = np.array(energy_bins)

print(np.sum(counts_thrumu))
print(np.sum(counts_hese))


In [ ]:
def beautify_counts(this_ax):
    sizer=25
    xlow =  1.e15 #the lower x limit
    xup = 1e21 #the uppper x limit
    this_ax.set_xlabel('Energy [eV]',size=sizer) #give it a title
    this_ax.set_ylabel('Events',size=sizer)
    this_ax.set_xscale('log')
    this_ax.tick_params(labelsize=sizer)
    # this_ax.set_xlim([xlow,xup]) #set the x limits of the plot
    this_ax.set_ylim([0,10]) #set the x limits of the plot
    this_ax.grid()
    this_legend = this_ax.legend(loc='upper left')#,title='Trigger-Level Event Counts')
    setp(this_legend.get_texts(), fontsize=20)
    setp(this_legend.get_title(), fontsize=20)
    this_legend.get_frame().set_edgecolor('k')
    # this_ax.xaxis.set_ticks([1e15, 1e17, 1e19, 1e21])
    # this_ax.yaxis.set_ticks([0.5, 1, 1.5, 2])

In [ ]:
fig2 = plt.figure(figsize=(2*8.5,2*3.5))
ax2_lim = fig2.add_subplot(1,2,1)
ax2_counts = fig2.add_subplot(1,2,2)
plot_bins = np.power(10.,np.arange(13,22,1))

plt.rc('text',usetex=True) #plot with Latex
plt.rc('font',family='serif') #Used a serif font
plt.rc('font', weight='heavy') #Make the text bold


n, bins, patches= ax2_counts.hist(energy_bins,
                                    bins=plot_bins,
                                    weights=counts_thrumu,
                                    # label=r'$\gamma=-2.37$',
                                    label=r'$\gamma=-2.37$: %.2f events'%counts_thrumu.sum(),
                                    fill=False, 
                                    histtype='step', 
                                    edgecolor='red',
                                    linewidth=4,
                                    linestyle='dashed')
n, bins, patches= ax2_counts.hist(energy_bins,
                                    bins=plot_bins,
                                    weights=counts_hese,
                                    label=r'$\gamma=-2.89$: %.2f events'%counts_hese.sum(),
                                    fill=False, 
                                    histtype='step', 
                                    edgecolor='blue',
                                    linewidth=4)
# ax2_counts.set_xscale('log')
beautify_counts(ax2_counts)

    